# CNN Attention Model

## 1. Imports, functions and constants

In [1]:
from CNN import *
from GradCAMUtils import *
from Utils_train import *

import numpy as np
import pandas as pd

import os

In [2]:
"""Constants"""
# sequence length indicate the maximum length for all of the sequnence 626/798
SEQUENCE_LENGTH = 798

BATCH_SIZE = 8

vocab = {'C': [0,0,1], 'H': [0,1,0], 'E': [1,0,0], '-':[0,0,0]}

# Transform the labels from String to Integer via LabelEncoder
le_fold = preprocessing.LabelEncoder()
le_fam = preprocessing.LabelEncoder()

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

cuda_gpu = torch.cuda.is_available()   #判断GPU是否存在可用

np.random.seed(2020)
torch.manual_seed(2020)
torch.cuda.manual_seed_all(2020)
torch.backends.cudnn.deterministic = True #make sure same results

## 2. Read datasets

In [3]:
# load all dataset
train_df = pd.read_csv("../Datasets/Training_data/gt_training.CNN/train_df.csv")
test_df = pd.read_csv("../Datasets/Training_data/gt_training.CNN/test_df.csv")
val_df = pd.read_csv("../Datasets/Training_data/gt_training.CNN/val_df.csv")

In [4]:
train_df.fold.value_counts()

A       60940
B       52697
C       19459
lyso     4932
u        3956
Name: fold, dtype: int64

In [5]:
#dump into MultiOutput Dataset
Train_ds, Train_dl = Dataset_LoaderTwo(train_df, le_fam, le_fold, vocab, BATCH_SIZE=BATCH_SIZE, cuda = cuda_gpu)

Test_ds, Test_dl = Dataset_LoaderTwo(test_df, le_fam, le_fold, vocab, BATCH_SIZE=BATCH_SIZE, cuda = cuda_gpu)

Val_ds, Val_dl = Dataset_LoaderTwo(val_df, le_fam, le_fold, vocab, BATCH_SIZE=BATCH_SIZE, cuda = cuda_gpu)

fold = train_df['fold'].unique()
family = train_df['family'].unique()

In [ ]:
# training the model

model = CNN_Attention(Fold=len(fold), Fam=len(family), Prob=0.5).cuda()

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss().cuda()
cnn_model = Fit(1, model, criterion, optimizer, Train_dl, Val_dl, patience=2, e=0.50)

In [ ]:
# pickle.dump(cnn_model,open('128_alldata_gtu.pickle','wb'))